# NLP Machine Learning Classifiers

## Building a Basic Random Forest Model

In [1]:
# Import the dependencies
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

### Read in & Clean Text Data

In [2]:
# Read in the text data
data = pd.read_csv("./data/SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

In [3]:
# Defind stopwords and Porter Stemmer
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

In [4]:
# Create count_punct() function
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

In [5]:
# Create clean_text() function
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

In [6]:
# Create new columns for body length and punctuation %
data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

In [7]:
# Clean the dataframe by TF-IDF Vectorizer
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

In [8]:
# Concat the dataframes
X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Explore Random Forest Classifier Attributes & Hyperparameters

In [9]:
from sklearn.ensemble import RandomForestClassifier

In [10]:
print(dir(RandomForestClassifier))
print(RandomForestClassifier())

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_estimator_type', '_get_param_names', '_get_tags', '_make_estimator', '_more_tags', '_required_parameters', '_set_oob_score', '_validate_X_predict', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params']
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0

### Explore Random Forest Classifier through Cross Validation

In [11]:
from sklearn.model_selection import KFold, cross_val_score

In [12]:
rf = RandomForestClassifier(n_jobs=-1)
k_fold = KFold(n_splits=5)
cross_val_score(rf, X_features, data['label'], cv=k_fold, scoring='accuracy', n_jobs=-1)

array([0.97217235, 0.97396768, 0.97394429, 0.95238095, 0.96675651])

## Building Random Forest Model with Grid-Search

**Grid-Search**: Exhaustively search all parameter combinations in a given grid to determine the best model.

### Building the Grid-Search

In [13]:
# Import the sklearn library
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [14]:
# Split the data into 80/20 raito for training and testing
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [15]:
# Create a function for training the random forest model
def train_RF(n_est, depth):
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
    rf_model = rf.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        n_est, depth, round(precision, 3), round(recall, 3),
        round((y_pred==y_test).sum() / len(y_pred), 3)))

In [16]:
# Use a for loop to calculate the precision, recall, and accuracy rate 
# with different combinations of parameters
for n_est in [10, 50, 100]:
    for depth in [10, 20, 30, None]:
        train_RF(n_est, depth)

Est: 10 / Depth: 10 ---- Precision: 1.0 / Recall: 0.236 / Accuracy: 0.901
Est: 10 / Depth: 20 ---- Precision: 1.0 / Recall: 0.597 / Accuracy: 0.948
Est: 10 / Depth: 30 ---- Precision: 0.991 / Recall: 0.743 / Accuracy: 0.966
Est: 10 / Depth: None ---- Precision: 0.991 / Recall: 0.778 / Accuracy: 0.97
Est: 50 / Depth: 10 ---- Precision: 1.0 / Recall: 0.25 / Accuracy: 0.903
Est: 50 / Depth: 20 ---- Precision: 1.0 / Recall: 0.542 / Accuracy: 0.941
Est: 50 / Depth: 30 ---- Precision: 1.0 / Recall: 0.701 / Accuracy: 0.961
Est: 50 / Depth: None ---- Precision: 1.0 / Recall: 0.785 / Accuracy: 0.972
Est: 100 / Depth: 10 ---- Precision: 1.0 / Recall: 0.264 / Accuracy: 0.905
Est: 100 / Depth: 20 ---- Precision: 1.0 / Recall: 0.576 / Accuracy: 0.945
Est: 100 / Depth: 30 ---- Precision: 1.0 / Recall: 0.729 / Accuracy: 0.965
Est: 100 / Depth: None ---- Precision: 1.0 / Recall: 0.792 / Accuracy: 0.973


## Evaluate Random Forest Model with Grid-Search Cross Validation

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Exploring parameter settings using GrdSearchCV

In [17]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [18]:
# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

In [19]:
# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

In [20]:
# Create the Random Forest Classifier and a set of parameters
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

# Create the Grid Search Cross Validation and a summary table for TF-IDT
gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,29.825429,0.327884,0.367870,0.066199,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.978475,0.975741,0.974843,0.968553,0.971249,0.973774,0.003486,1
10,30.475670,0.753745,0.383542,0.112190,None,150,"{'max_depth': None, 'n_estimators': 150}",0.975785,0.977538,0.974843,0.969452,0.970350,0.973594,0.003149,2
11,51.395380,0.505338,0.347755,0.014461,None,300,"{'max_depth': None, 'n_estimators': 300}",0.978475,0.977538,0.973046,0.967655,0.971249,0.973594,0.004012,2
4,23.337060,0.706488,0.380976,0.021750,60,150,"{'max_depth': 60, 'n_estimators': 150}",0.978475,0.976640,0.973046,0.967655,0.970350,0.973235,0.003964,4
8,56.195443,1.646959,0.733871,0.112254,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.979372,0.974843,0.972147,0.967655,0.972147,0.973235,0.003842,4


In [ ]:
# Create the Random Forest Classifier and a set of parameters
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

# Create the Grid Search Cross Validation and a summary table for Count Vectorizer
gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

## Explore Gradient Boosting Model with Grid-Search

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

### Explore Gradient Boosting Classifier Attributes and Hyperparameters

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
print(dir(GradientBoostingClassifier))
print(GradientBoostingClassifier())

### Building the Grid-Search

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [ ]:
# Split the data into 80/20 raito for training and testing
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [ ]:
# Create function for training with Gradient Boosting
def train_GB(est, max_depth, lr):
    gb = GradientBoostingClassifier(n_estimators=est, max_depth=max_depth, learning_rate=lr)
    gb_model = gb.fit(X_train, y_train)
    y_pred = gb_model.predict(X_test)
    precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} / LR: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        est, max_depth, lr, round(precision, 3), round(recall, 3), 
        round((y_pred==y_test).sum()/len(y_pred), 3)))

In [ ]:
# Create for loop to run the model with different parameters
for n_est in [50, 100, 150]:
    for max_depth in [3, 7, 11, 15]:
        for lr in [0.01, 0.1, 1]:
            train_GB(n_est, max_depth, lr)

## Evaluate Gradient Boosting with Grid-Search Cross Validation

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Exploring parameter settings using GridSearchCV

In [ ]:
# Import the dependencies
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
# Create the Gradient Boosting Classifier and a set of parameters
gb = GradientBoostingClassifier()
param = {'n_estimators': [100, 150], 
    'max_depth': [7, 11, 15], 'learning_rate': [0.1]}

# Create the Grid Search Cross Validation and a summary table for TF-IDF
clf = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = clf.fit(X_tfidf_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

In [ ]:
# Create the Gradient Boosting Classifier and a set of parameters
gb = GradientBoostingClassifier()
param = {'n_estimators': [50, 100, 150], 
    'max_depth': [7, 11, 15], 'learning_rate': [0.1]}

# Create the Grid Search Cross Validation and a summary table for Count Vectorizer
clf = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = clf.fit(X_count_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

## Model Selection

### Split the Data into Training and Testing Set

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data[['body_text', 'body_len', 'punct%']], data['label'], test_size=0.2)

### Vectorize Text

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
tfidf_vect_fit = tfidf_vect.fit(X_train['body_text'])

tfidf_train = tfidf_vect_fit.transform(X_train['body_text'])
tfidf_test = tfidf_vect_fit.transform(X_test['body_text'])

X_train_vect = pd.concat([X_train[['body_len', 'punct%']].reset_index(drop=True), 
           pd.DataFrame(tfidf_train.toarray())], axis=1)
X_test_vect = pd.concat([X_test[['body_len', 'punct%']].reset_index(drop=True), 
           pd.DataFrame(tfidf_test.toarray())], axis=1)

X_train_vect.head()

### Final Evaluation of Models

In [ ]:
# Import the dependencies
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time

In [ ]:
# Evaluate the Random Forest Model
rf = RandomForestClassifier(n_estimators=150, max_depth=None, n_jobs=-1)

rf_model = rf.fit(X_train_vect, y_train)
y_pred = rf_model.predict(X_test_vect)

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

In [ ]:
# Evaluate the Gradient Boosting Model
gb = GradientBoostingClassifier(n_estimators=150, max_depth=11)

gb_model = gb.fit(X_train_vect, y_train)
y_pred = gb_model.predict(X_test_vect)

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))